In [25]:
import re

array_contig = []
counter = 0
with open("out_contig.fa") as f:
    for line in f:
        if line[0] == ">":
            array_contig.append(int(line.split("_")[1][3:]))
array_contig.sort(reverse=True)
total_contig_length = sum(array_contig)
total_contig_num = len(array_contig)
max_contig = array_contig[0]
for i in array_contig:
    counter += i
    if counter >= total_contig_length // 2:
        n50 = i
        break
print("Общее количество контигов: {}, их общая длина: {}, длина самого длинного контига: {}, N50: {}".format(total_contig_num, total_contig_length, max_contig, n50))

array_scaffold = []
counter = 0
with open("out_scaffold.fa") as f:
    for line in f:
        if line[0] == ">":
            array_scaffold.append(int(line.split("_")[1][3:]))
array_scaffold.sort(reverse=True)
total_scaffold_length = sum(array_scaffold)
total_scaffold_num = len(array_scaffold)
max_scaffold = array_scaffold[0]
for i in array_scaffold:
    counter += i
    if counter >= total_scaffold_length // 2:
        n50 = i
        break
print("Общее количество скаффлдов: {}, их общая длина: {}, длина самого длинного скаффолда: {}, N50: {}".format(total_scaffold_num, total_scaffold_length, max_scaffold, n50))

with open("longest.fa") as f:
    all = f.read()
    nnn = re.findall('N{1,}', all)
    print("Общее количество гэпов (not gap closed): {}, их общая длина: {}".format(len(nnn), sum([len(i) for i in nnn])))

with open("longest_closed.fa") as f:
    all = f.read()
    nnn = re.findall('N{1,}', all)
    print("Общее количество гэпов (gap closed): {}, их общая длина: {}".format(len(nnn), sum([len(i) for i in nnn])))

Общее количество контигов: 630, их общая длина: 3926892, длина самого длинного контига: 179304, N50: 53978
Общее количество скаффлдов: 71, их общая длина: 3875743, длина самого длинного скаффолда: 3832127, N50: 3832127
Общее количество гэпов (not gap closed): 68, их общая длина: 6988
Общее количество гэпов (gap closed): 10, их общая длина: 1848
